# Generate object(sign) and annotations on draw

In [1]:
%matplotlib inline

import os
from glob import glob
from matplotlib import pyplot
from PIL import Image
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2

In [2]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')

# Reshape Sign 
size = 15, 15

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im.save(s)

# Change color of sign

In [67]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = inputImg
    data = np.array(im)

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)
    im.save('./data/backGround/testSign.png')
    
    return im

# Make n-Sign on Draw with loaction(x,y) information 

In [68]:

def genData(clss, amount, dr_fileName,r2,g2,b2):
    
    l_img =  cv2.imread(dr_fileName)
    s_img =  cv2.imread('./data/backGround/sign/'+clss+'.png',-1) 
    
    xx = s_img.shape[1]
    yy = s_img.shape[0]
    
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    dr_fileName = dr_fileName.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    
    new_dirFileName = './data/draw/dataset/GN_'+dr_fileName+str(random.random()*6)+'.png'   
        
    new_fileName  = new_dirFileName .split('dataset/')[1]
    
    
    s_img = ChangeColor(s_img,r2,g2,b2)
    
    h, w, d =l_img.shape
    
    # make annotations file
    root = etree.Element("annotation")

    folder = etree.SubElement(root, "folder").text ='draw'  
    filename =  etree.SubElement(root, "filename").text = new_fileName    
    path =  etree.SubElement(root, "path").text = new_dirFileName

    source =  etree.SubElement(root, "source")
    database =  etree.SubElement(source, "database").text= 'Unknown'     
    
    size =  etree.SubElement(root, "size")
    width =  etree.SubElement(size, "width").text = str(w)
    height =  etree.SubElement(size, "height").text = str(h)
    depth =  etree.SubElement(size, "depth").text = str(d)

    segmented =  etree.SubElement(root, "segmented").text = '0'
    
    
    
    # generate signs and annotaion files.                 
    for i in range(amount):    
               
        xmin = random.sample(range(100,w),1)
        ymin = random.sample(range(100,h),1)
                
        x_offset=y_offset=16
        
        x1, x2 = xmin, x_offset + xx
        y1, y2 = ymin, y_offset + yy

        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clss
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'
        
        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
    
    tree = etree.ElementTree(root)
    with open('./data/backGround/'+new_fileName+'.xml', "bw") as fh:
        tree.write(fh)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

    
    cv2.imwrite(new_fileName, l_img)   

In [69]:
genData('DT_150',10, './data/backGround/draw/b1.png',255,228,0)

TypeError: 'Image' object is not subscriptable